In [1]:
from typing import List, Union
import math

In [2]:
class Vector:
    def __init__(self, data: List[float]):
        self.data: List[float] = data
        self.size: int = len(data)


    def __add__(self, other: 'Vector') -> 'Vector':
        assert isinstance(other, Vector), "Values must be instances of Vector"
        assert self.size == other.size, "Vectors must have the same size for addition"
        return Vector([x + y for x, y in zip(self.data, other.data)])
   
   
    def __sub__(self, other: 'Vector') -> 'Vector':
        assert isinstance(other, Vector), "Values must be instances of Vector"
        assert self.size == other.size, "Vectors must have the same size for subtraction"
        return Vector([x - y for x, y in zip(self.data, other.data)])


    def __mul__(self, scalar: Union[int, float]) -> 'Vector':
        return Vector([scalar * x for x in self.data])


    def __rmul__(self, scalar: Union[int, float]) -> 'Vector':
        return self * scalar


    def __getitem__(self, index: Union[int, slice]) -> float:
        return self.data[index]


    def __str__(self) -> str:
        return f"Vector({self.data})"


    def __repr__(self) -> str:
        return str(self)


    def dot(self, other: 'Vector') -> Union[int, float]:
        assert isinstance(other, Vector), "Values must be instances of Vector"
        assert self.size == other.size, "Vectors must have the same size for dot product"
        return sum([x * y for x, y in zip(self.data, other.data)])


    def cross(self, other: 'Vector') -> 'Vector':
        assert isinstance(other, Vector), "Values must be instances of Vector"
        assert self.size == other.size == 3, "Cross product is defined only for 3-dimensional vectors"
        return Vector([self.data[1] * other.data[2] - self.data[2] * other.data[1],
                       self.data[2] * other.data[0] - self.data[0] * other.data[2],
                       self.data[0] * other.data[1] - self.data[1] * other.data[0]])


    def magnitude(self) -> Union[int, float]:
        return sum(x ** 2 for x in self.data) ** 0.5


    def normalize(self) -> 'Vector':
        mag = self.magnitude()
        if mag == 0:
            return Vector([0] * self.size)
        else:
            return Vector([x / mag for x in self.data])


    @classmethod
    def zero(cls, size: int) -> 'Vector':
        return Vector([0] * size)


    def is_zero(self) -> bool:
        for value in self.data:
            if value != 0:
                return False
        return True


    def unit(self) -> 'Vector':
        return self.normalize()


    def is_unit(self) -> bool:
        return math.isclose(self.magnitude(), 1)


    def angle(self, other: 'Vector') -> float:
        assert not self.is_zero() and not other.is_zero(), "Vectors must not be zeros"
        return math.acos(self.dot(other) / (self.magnitude() * other.magnitude()))


    def is_parallel(self, other: 'Vector') -> bool:
        return math.isclose(self.angle(other), 0)


    def is_orthogonal(self, other: 'Vector') -> bool:
        return math.isclose(self.dot(other), 0)


    def distance(self, other: 'Vector') -> float:
        return (self - other).magnitude()


In [3]:
class Matrix:
    def __init__(self, data: List[List[float]]):
        assert all(len(row) == len(data[0]) for row in data), "All rows must have the same number of elements"
        self.data: List[List[float]] = data
        self.rows: int = len(data)
        self.columns: int = len(data[0])
        self.shape: tuple = self.rows, self.columns

    def __add__(self, other: 'Matrix') -> 'Matrix':
        assert isinstance(other, Matrix), "Values must be instances of Matrix"
        assert (self.rows, self.columns) == (other.rows, other.columns), "Matrices must have the same size for addition"
        return Matrix([[ai + bi for ai, bi in zip(a, b)] for a, b in zip(self.data, other.data)])


    def __sub__(self, other: 'Matrix') -> 'Matrix':
        assert isinstance(other, Matrix), "Values must be instances of Matrix"
        assert (self.rows, self.columns) == (other.rows, other.columns), "Matrices must have the same size for subtraction"
        return Matrix([[ai - bi for ai, bi in zip(a, b)] for a, b in zip(self.data, other.data)])


    def __mul__(self, other: Union['Matrix', int, float]) -> Union['Matrix', 'Vector']:
        assert isinstance(other, (Matrix, int, float, Vector)), "Values must be instances of Matrix, Vector or scalars"
        if isinstance(other, (int, float)):
            return Matrix([[ai * other for ai in a] for a in self.data])
        
        result = []
        if isinstance(other, Vector):
            assert self.columns == other.size, "Number of columns matrix must match the vector size"
            for i in range(self.rows):
                value = sum(self.data[i][k] * other.data[k] for k in range(self.columns))
                result.append(value)
            return Vector(result)
            
        assert self.columns == other.rows, "Number of columns in the first matrix must match the number of rows in the second matrix"
        for i in range(self.rows):
            row = []
            for j in range(other.columns):
                value = sum(self.data[i][k] * other.data[k][j] for k in range(self.columns))
                row.append(value)
            result.append(row)
        return Matrix(result)
    
    
    def __rmul__(self, other: Union['Matrix', int, float]) -> 'Matrix':
        return self * other


    def __eq__(self, other: 'Matrix') -> bool:
        assert isinstance(other, Matrix), "Values must be instances of Matrix"
        return self.data == other.data

    
    def __ne__(self, other: 'Matrix') -> bool:
        return not self == other


    def __getitem__(self, index: Union[int, tuple[int, int]]) -> Union[float, 'Vector']:
        assert isinstance(index, (int, tuple)), "Indexes must be integers or tuples of integers"
        if isinstance(index, int):
            return Vector(data=self.data[index])
        elif isinstance(index, tuple):
            assert isinstance(index[0], (int, slice)), "First index must be an integer or slice"
            assert isinstance(index[1], (int, slice)), "Second index must be an integer or slice"
            if isinstance(index[0], int) and isinstance(index[1], int):
                return self.data[index[0]][index[1]]
            elif isinstance(index[0], int) and isinstance(index[1], slice):
                return Vector(data=self.data[index[0]][index[1]])
            elif isinstance(index[0], slice) and isinstance(index[1], int):
                return Vector(data=[row[index[1]] for row in self.data[index[0]]])


    def __setitem__(self, index: Union[int, tuple[int, int]], value: Union[float, int, 'Vector', List[float]]):
        assert isinstance(index, (int, tuple)), "Indexes must be integers or tuples of integers"
        if isinstance(index, int):
            if isinstance(value, (float, int)):
                self.data[index] = value
            elif isinstance(value, Vector):
                self.data[index] = value.data
            elif isinstance(value, list):
                assert all(isinstance(x, (float, int)) for x in value), "List elements must be floats"
                self.data[index] = value
            else:
                raise ValueError("Value must be a float, Vector, or list of floats")
        elif isinstance(index, tuple):
            assert isinstance(index[0], (int, slice)), "First index must be an integer or slice"
            assert isinstance(index[1], (int, slice)), "Second index must be an integer or slice"
            if isinstance(index[0], int) and isinstance(index[1], int):
                if isinstance(value, (float, int)):
                    self.data[index[0]][index[1]] = value
                elif isinstance(value, Vector):
                    self.data[index[0]][index[1]] = value.data
                elif isinstance(value, list):
                    assert all(isinstance(x, (float, int)) for x in value), "List elements must be floats"
                    self.data[index[0]][index[1]] = value
                else:
                    raise ValueError("Value must be a float, Vector, or list of floats")
            elif isinstance(index[0], int) and isinstance(index[1], slice):
                if isinstance(value, Vector):
                    self.data[index[0]][index[1]] = value.data
                elif isinstance(value, list):
                    assert all(isinstance(x, (float, int)) for x in value), "List elements must be floats"
                    self.data[index[0]][index[1]] = value
                else:
                    raise ValueError("Value must be a Vector or list of floats")
            elif isinstance(index[0], slice) and isinstance(index[1], int):
                if isinstance(value, Vector):
                    assert len(value.data) == len(range(*index[0].indices(self.rows))), "Vector size must match the slice length"
                    for i, row_index in enumerate(range(*index[0].indices(self.rows))):
                        self.data[row_index][index[1]] = value.data[i]
                elif isinstance(value, list):
                    assert all(isinstance(x, (float, int)) for x in value), "List elements must be floats"
                    assert len(value) == len(range(*index[0].indices(self.rows))), "List length must match the slice length"
                    for i, row_index in enumerate(range(*index[0].indices(self.rows))):
                        self.data[row_index][index[1]] = value[i]
                else:
                    raise ValueError("Value must be a Vector or list of floats")


    def __str__(self) -> str:
        return f"Matrix({self.data})"


    def __repr__(self) -> str:
        return str(self)


    def transpose(self) -> 'Matrix':
        result = [[self.data[j][i] for j in range(self.rows)] for i in range(self.columns)]
        return Matrix(result)


    def row(self, index: int) -> 'Vector':
        return self[index]


    def column(self, index: int) -> 'Vector':
        return self[:, index]


    def trace(self) -> Union[int, float]:
        assert self.is_square(), "Matrix must be square to compute trace"
        return sum(self.data[i][i] for i in range(self.rows))


    def minor(self, i: int, j: int) -> 'Matrix':
        return Matrix([row[:j] + row[j + 1:] for row in (self.data[:i] + self.data[i + 1:])])


    def determinant(self) -> Union[int, float]:
        assert self.is_square(), "Matrix must be square to compute determinant"
        if self.rows == 1:
            return self.data[0][0]
        if self.rows == 2:
            return self.data[0][0] * self.data[1][1] - self.data[0][1] * self.data[1][0]
        
        if self.is_diagonal() or self.is_upper_triangular() or self.is_lower_triangular():
            det = 1
            for i in range(self.rows):
                det *= self.data[i][i]
            return det
               
        det = sum((-1) ** j * self.data[0][j] * self.minor(0, j).determinant() for j in range(self.columns))
        return det


    def inverse(self) -> 'Matrix':
        assert self.is_square(), "Matrix must be square to compute inverse"
        
        det = self.determinant()
        assert det != 0, "The matrix is not invertible"
        
        inverse = Matrix.zero(self.rows, self.columns)

        for i in range(self.rows):
            for j in range(self.columns):
                cofactor_ij = (-1) ** (i + j) * self.minor(i, j).determinant()
                inverse.data[j][i] = cofactor_ij / det

        return inverse



    @classmethod
    def identity(cls, size: int) -> 'Matrix':
        return Matrix([[1 if i == j else 0 for j in range(size)] for i in range(size)])


    @classmethod
    def zero(cls, rows: int, columns: int) -> 'Matrix':
        return Matrix([[0] * columns for _ in range(rows)])


    def is_square(self) -> bool:
        return self.rows == self.columns


    def is_symmetric(self) -> bool:
        return self == self.transpose()
    
    
    def is_orthogonal(self) -> bool:
        return self.inverse() == self.transpose()

    
    def is_diagonal(self) -> bool:
        assert self.is_square(), "Matrix must be square first"
        for i in range(self.rows):
            for j in range(self.columns):
                if i != j:
                    if self.data[i][j] != 0:
                        return False
        return True


    def is_upper_triangular(self) -> bool:
        assert self.is_square(), "Matrix must be square first"
        for i in range(1, self.rows):
            for j in range(i):
                if self.data[i][j] != 0:
                    return False
        return True


    def is_lower_triangular(self) -> bool:
        assert self.is_square(), "Matrix must be square first"
        for i in range(0, self.rows - 1):
            for j in range(i+1, self.columns):
                if self.data[i][j] != 0:
                    return False
        return True
    
    
    def solve_by_forward_substitution(self, vector: 'Vector'):
        assert vector.size == self.columns, "Vector size must match the number of columns"
        assert self.is_lower_triangular(), "Forward substitution requires the matrix to be lower triangular"
        solution = []
        for row_index in range(vector.size):
            assert self.data[row_index][row_index] != 0, "Singular matrix: No unique solution exists"
            solution_value = (vector[row_index] - sum(self.data[row_index][column_index] * solution[column_index] for column_index in range(row_index))) / self.data[row_index][row_index]
            solution.append(solution_value)
        return Vector(solution)


    def solve_by_backward_substitution(self, vector: 'Vector'):
        assert vector.size == self.columns, "Vector size must match the number of columns"
        assert self.is_upper_triangular(), "Backward substitution requires the matrix to be upper triangular"
        solution = [None] * vector.size
        for row_index in range(vector.size - 1, -1, -1):
            assert self.data[row_index][row_index] != 0, "Singular matrix: No unique solution exists"
            solution_value = vector[row_index]
            for column_index in range(row_index + 1, vector.size):
                solution_value -= self.data[row_index][column_index] * solution[column_index]
            solution[row_index] = solution_value / self.data[row_index][row_index]
        return Vector(solution)
    
    
    def copy(self):
        return Matrix([row.copy() for row in self.data])
    
    
    def LU(self):
        assert self.is_square(), "Matrix must be square first"
        u = self.copy()
        l = Matrix.identity(self.rows)
        for j in range(self.columns):
            assert u[j, j] != 0, "Matrix either requires PLU or is Singular"
            for i in range(j+1, self.rows):
                l[i, j] = u[i, j] / u[j, j]
                u[i, j:] -= l[i, j] * u[j, j:]
        return l, u


    def PLU(self):
        assert self.is_square(), "Matrix must be square first"
        u = self.copy()
        l = Matrix.identity(self.rows)
        p = Matrix.identity(self.rows)
        for j in range(self.columns):
            if u[j, j] == 0:
                permuted = False
                for k in range(j, self.rows):
                    if u[k, j] != 0:
                        u[j], u[k] = u[k], u[j]
                        p[j, j], p[k, k] = 0, 0
                        p[k, j], p[j, k] = 1, 1
                        permuted = True
                        break
                assert permuted, "Singular matrix: LU decomposition failed"
            
                
            for i in range(j+1, self.rows):
                l[i, j] = u[i, j] / u[j, j]
                u[i, j:] -= l[i, j] * u[j, j:]
            
        return p, l, u
    
    
    def compute_det_lu(self):
        assert self.is_square(), "Matrix must be square first"
        u = self.copy()
        l = Matrix.identity(self.rows)
        p = 0
        for j in range(self.columns):
            if u[j, j] == 0:
                permuted = False
                for k in range(j, self.rows):
                    if u[k, j] != 0:
                        u[j], u[k] = u[k], u[j]
                        p += 1
                        permuted = True
                        break
                if not permuted:
                    return 0
                     
            for i in range(j+1, self.rows):
                l[i, j] = u[i, j] / u[j, j]
                u[i, j:] -= l[i, j] * u[j, j:]
        
        det = (-1)**p
        for i in range(self.rows):
            det *= u[i, i]
        
        return det
    
    
    def compute_inverse_lu(self):
        assert self.is_square(), "Matrix must be square first"
        u = self.copy()
        l = Matrix.identity(self.rows)
        p = Matrix.identity(self.rows)
        for j in range(self.columns):
            if u[j, j] == 0:
                permuted = False
                for k in range(j, self.rows):
                    if u[k, j] != 0:
                        u[j], u[k] = u[k], u[j]
                        p[j, j], p[k, k] = 0, 0
                        p[k, j], p[j, k] = 1, 1
                        permuted = True
                        break
                assert permuted, "Matrix is singular (zero pivot encountered)"   
                     
            for i in range(j+1, self.rows):
                l[i, j] = u[i, j] / u[j, j]
                u[i, j:] -= l[i, j] * u[j, j:]
        
        inverse = Matrix.zero(self.rows, self.columns)

        
        for i in range(self.rows):
            vector = p.column(i)
            first = l.solve_by_forward_substitution(vector)
            second = u.solve_by_backward_substitution(first)
            inverse[:, i] = second
        
        return inverse
    
    
    def solve_by_lu(self, vector: 'Vector'):
        assert self.is_square(), "Matrix must be square first"
        u = self.copy()
        l = Matrix.identity(self.rows)
        p = Matrix.identity(self.rows)
        for j in range(self.columns):
            if u[j, j] == 0:
                permuted = False
                for k in range(j, self.rows):
                    if u[k, j] != 0:
                        u[j], u[k] = u[k], u[j]
                        p[j, j], p[k, k] = 0, 0
                        p[k, j], p[j, k] = 1, 1
                        permuted = True
                        break
                assert permuted, "Singular matrix: No unique solution exists"   
                     
            for i in range(j+1, self.rows):
                l[i, j] = u[i, j] / u[j, j]
                u[i, j:] -= l[i, j] * u[j, j:]
        
        b = p*vector
        first = l.solve_by_forward_substitution(b)
        second = u.solve_by_backward_substitution(first)
        
        return second
    
    
    def cholesky(self):
        assert self.is_symmetric(), "The matrix must be symmetric to do Cholesky's decomposition"
        l = Matrix.zero(self.rows, self.columns)
        for i in range(self.rows):
            for j in range(i+1):
                s = 0
                for k in range(j):
                    s += l[i, k] * l[j, k]
                
                value = self[i, j] - s
                
                if (i == j):
                    assert value > 0, "The matrix must be positive-definite to do Cholesky's decomposition"
                    l[i, j] = value ** 0.5
                else:
                    l[i, j] = value / l[j, j]
            
        return l        


    def solve_by_cholesky(self, vector: 'Vector'):
        assert self.is_symmetric(), "The matrix must be symmetric to do Cholesky's decomposition"
        l = Matrix.zero(self.rows, self.columns)
        for i in range(self.rows):
            for j in range(i+1):
                s = 0
                for k in range(j):
                    s += l[i, k] * l[j, k]
                
                value = self[i, j] - s
                
                if (i == j):
                    assert value > 0, "The matrix must be positive-definite to do Cholesky's decomposition"
                    l[i, j] = value ** 0.5
                else:
                    l[i, j] = value / l[j, j]
        
        first = l.solve_by_forward_substitution(vector)
        second = l.transpose().solve_by_backward_substitution(first)        
        
        return second

In [4]:
(Matrix([[1], [2], [3], [4]]) * Matrix([[1, 2, 3, 4]])).shape

(4, 4)

In [5]:
# Create some matrices for testing
matrix1 = Matrix([[1, 2], [3, 4]])
matrix2 = Matrix([[5, 6], [7, 8]])
matrix3 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
matrix4 = Matrix([[2, 0, 0], [0, 3, 0], [0, 0, 4]])
matrix5 = Matrix([[1, 2], [3, 4], [5, 6]])
matrix6 = Matrix([[1, 0], [0, 1]])

In [6]:
matrix1 + matrix2

Matrix([[6, 8], [10, 12]])

In [7]:
matrix2 - matrix1

Matrix([[4, 4], [4, 4]])

In [8]:
matrix1 * 2

Matrix([[2, 4], [6, 8]])

In [9]:
matrix1 * matrix2

Matrix([[19, 22], [43, 50]])

In [10]:
matrix3.transpose()

Matrix([[1, 4, 7], [2, 5, 8], [3, 6, 9]])

In [11]:
matrix3.trace()

15

In [12]:
matrix4.determinant()

24

In [13]:
3 * matrix1

Matrix([[3, 6], [9, 12]])

In [14]:
matrix6.inverse()

Matrix([[1.0, 0.0], [0.0, 1.0]])

In [15]:
Matrix.identity(3)

Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [16]:
Matrix.zero(2, 3)

Matrix([[0, 0, 0], [0, 0, 0]])

In [17]:
matrix1.is_square(), matrix5.is_square()

(True, False)

In [18]:
matrix1.is_symmetric(), matrix6.is_symmetric()

(False, True)

In [19]:
matrix7 = Matrix([[4, 3, 2, 2], [0, 1, -3, 3], [0, -1, 3, 3], [0, 3, 1, 1]])

In [20]:
matrix7

Matrix([[4, 3, 2, 2], [0, 1, -3, 3], [0, -1, 3, 3], [0, 3, 1, 1]])

In [21]:
matrix7.determinant()

-240

In [22]:
Matrix.identity(6).inverse() == Matrix.identity(6)

True

In [23]:
Matrix.identity(6).is_upper_triangular()

True

In [24]:
matrix1.is_diagonal()

False

In [25]:
matrix8 = Matrix([[1, 2, 3],
                  [0, 4, 5],
                  [0, 0, 6]])

matrix8.is_diagonal(), matrix8.is_upper_triangular(), matrix8.is_lower_triangular()

(False, True, False)

In [26]:
matrix9 = Matrix([[1, 0, 0],
                  [4, 5, 0],
                  [7, 8, 9]])

matrix9.is_diagonal(), matrix9.is_upper_triangular(), matrix9.is_lower_triangular()

(False, False, True)

In [27]:
matrix9[0, 0]

1

In [28]:
matrix9[:, 1]

Vector([0, 5, 8])

In [29]:
matrix9.column(1)

Vector([0, 5, 8])

In [30]:
matrix10 = Matrix([[3, 0, 0, 0], [-1, 1, 0, 0], [3, -2, -1, 0], [1, -2, 6, 2]])
vector10 = Vector([5, 6, 4, 2])

In [31]:
matrix10.solve_by_forward_substitution(vector10)

Vector([1.6666666666666667, 7.666666666666667, -14.333333333333334, 50.833333333333336])

In [32]:
list(reversed([list(reversed(i)) for i in matrix10.data]))

[[2, 6, -2, 1], [0, -1, -2, 3], [0, 0, 1, -1], [0, 0, 0, 3]]

In [33]:
matrix11 = Matrix(list(reversed([list(reversed(i)) for i in matrix10.data])))
vector11 = Vector(vector10.data[::-1])
matrix11.solve_by_backward_substitution(vector11)

Vector([50.833333333333336, -14.333333333333336, 7.666666666666667, 1.6666666666666667])

In [34]:
I3 = Matrix.identity(3)
I3[:, 0] = [1, 7, 4]
I3

Matrix([[1, 0, 0], [7, 1, 0], [4, 0, 1]])

In [35]:
I3[0] = [2]*3

In [36]:
I3[0, 1:] = [3, 4]

In [37]:
I3

Matrix([[2, 3, 4], [7, 1, 0], [4, 0, 1]])

In [38]:
matrix12 = Matrix([[2, 1, 1, 0], [4, 3, 3, 1], [8, 7, 9, 5], [6, 7, 9, 8]])

In [39]:
l, u = matrix12.LU()
l, u

(Matrix([[1, 0, 0, 0], [2.0, 1, 0, 0], [4.0, 3.0, 1, 0], [3.0, 4.0, 1.0, 1]]),
 Matrix([[2, 1, 1, 0], [0.0, 1.0, 1.0, 1.0], [0.0, 0.0, 2.0, 2.0], [0.0, 0.0, 0.0, 2.0]]))

In [40]:
matrix12

Matrix([[2, 1, 1, 0], [4, 3, 3, 1], [8, 7, 9, 5], [6, 7, 9, 8]])

In [41]:
matrix12.determinant(), l.determinant()*u.determinant()

(8, 8.0)

In [42]:
l*u - matrix12

Matrix([[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]])

In [43]:
matrix13 = Matrix([[0, 5, 22/3], [4, 2, 1], [2, 7, 9]])

In [44]:
p, l, u = matrix13.PLU()
p, l, u

(Matrix([[0, 1, 0], [1, 0, 0], [0, 0, 1]]),
 Matrix([[1, 0, 0], [0.0, 1, 0], [0.5, 1.2, 1]]),
 Matrix([[4, 2, 1], [0.0, 5.0, 7.333333333333333], [0.0, 0.0, -0.29999999999999893]]))

In [45]:
p.transpose()*l*u - matrix13

Matrix([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [46]:
p.transpose() - Matrix.identity(3)

Matrix([[-1, 1, 0], [1, -1, 0], [0, 0, 0]])

In [47]:
matrix13.determinant()

6.0

In [48]:
matrix13.compute_det_lu()

5.999999999999979

In [49]:
math.isclose(matrix13.determinant(), matrix13.compute_det_lu())

True

In [50]:
matrix13.compute_inverse_lu()*matrix13

Matrix([[1.0, 0.0, 5.329070518200751e-15], [0.0, 0.9999999999999929, -7.105427357601002e-15], [0.0, 3.552713678800501e-15, 1.0]])

In [51]:
matrix13.compute_inverse_lu()

Matrix([[1.8333333333333401, 1.0555555555555582, -1.6111111111111165], [-5.666666666666687, -2.4444444444444526, 4.888888888888905], [4.000000000000014, 1.6666666666666725, -3.333333333333345]])

In [52]:
matrix13.inverse()

Matrix([[1.8333333333333333, 1.0555555555555547, -1.611111111111111], [-5.666666666666667, -2.444444444444444, 4.888888888888888], [4.0, 1.6666666666666667, -3.3333333333333335]])

In [53]:
matrix13.solve_by_lu(Vector([1, 0, 0]))

Vector([1.8333333333333401, -5.666666666666687, 4.000000000000014])

In [54]:
matrix14 = Matrix([[16, -8, -4], [-8, 29, 12], [-4, 12, 41]])

In [55]:
l = matrix14.cholesky()
l

Matrix([[4.0, 0, 0], [-2.0, 5.0, 0], [-1.0, 2.0, 6.0]])

In [56]:
l*l.transpose() - matrix14

Matrix([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [57]:
vector14 = Vector([1, 2, 3])

In [58]:
matrix14.solve_by_cholesky(vector14)

Vector([0.115625, 0.075, 0.0625])

In [59]:
matrix14.solve_by_lu(Vector([1, 2, 3]))

Vector([0.115625, 0.075, 0.0625])

In [60]:
matrix14 * Vector([0.115625, 0.075, 0.0625])

Vector([1.0, 1.9999999999999998, 3.0])